In [16]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers


######################################################################
######################################################################
######################################################################
## CSP File from: https://github.com/spolsley/common-spatial-patterns

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])
			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)

	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)
######################################################################
def GetMinSteps(indeces, data):
	minVal = 9999;
	for index in indeces:
		length = data[index].shape[1];
		if(length < minVal):
			minVal = length;
	return minVal
######################################################################
def GetData(indeces, dataIn, truncateValue):
	dataOut = []
	truncate = True;
	if truncateValue == 0:
		truncate = False;
	for idx in indeces:
		currentData = dataIn[idx]
		if truncate:
			dataOut.append(currentData[:,0:truncateValue])
		else:
			dataOut.append(currentData)
	return np.asarray(dataOut)
######################################################################
######################################################################
######################################################################

fileNames = ['CLASubjectA1601083StLRHand.mat',\
             'CLASubjectB1510193StLRHand.mat',\
             'CLASubjectB1510203StLRHand.mat',\
             'CLASubjectB1512153StLRHand.mat',\
             'CLASubjectC1511263StLRHand.mat',\
             'CLASubjectC1512163StLRHand.mat',\
             'CLASubjectC1512233StLRHand.mat',\
             'CLASubjectD1511253StLRHand.mat',\
             'CLASubjectE1512253StLRHand.mat',\
             'CLASubjectE1601193StLRHand.mat',\
             'CLASubjectE1601223StLRHand.mat',\
             'CLASubjectF1509163StLRHand.mat',\
             'CLASubjectF1509173StLRHand.mat',\
             'CLASubjectF1509283StLRHand.mat'];

numDatasets = len(fileNames);

testAccuracyNoCSP = np.zeros((1,numDatasets))
trainAccuracyNoCSP = np.zeros((1,numDatasets))
testAccuracyWithCSP = np.zeros((1,numDatasets))
trainAccuracyWithCSP = np.zeros((1,numDatasets))

for dataset in range(0, numDatasets):

	fileName = fileNames[dataset];

	file = sio.loadmat('../../../matDown/CLA_Data/{}'.format(fileName)) #replace with .mat file name
	header=file['__header__']
	version=file['__version__']
	glob=file['__globals__']
	#ans=file['ans']


	#x=file['x']
	o=file['o'][0][0]
	data=o['data']
	data = np.transpose(data)
	data = data[0:21,:];
	print(data)
	nS=o['nS'][0][0]
	#values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
	#use [0][0] to get scalar.
	#print("Number of samples: {numSamples}".format(numSamples=nS))
	test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
	#print("Dataset ID: {id}".format(id=test))
	chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
	#print("Channel names: {channelNames}".format(channelNames=chnames))
	markers = o['marker']
	## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
	markersArray = []
	for marker in markers:
		markersArray.append(marker[0])
	markersArray = np.asarray(markersArray)

	#################################
	#################################
	#5F interaction paradigm
	#1-thumb MI, 2-index finger MI, 3-middle finger MI, 4-ring finger MI, 5-pinkie finger MI

	#all paradigms
	#99-initial relaxation period
	#91-inter-session rest break period
	#92-experiment end
	#################################

	## Find the starting indeces where the marker changes
	changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
	#print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
	## Split the data so that it has its matching marker
	dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
	splitCount = 0
	for splitData in dataSplit:
		splitCount += 1
	#print("Number of arrays in data split: {num}".format(num=splitCount))
	## Retrieve the marker values for each of the change indeces (changeIdxs)
	markerTargets = markersArray[changeIdxs];
	#print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

	## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
	lhIdx = np.where(markerTargets == 1)[0]
	rhIdx = np.where(markerTargets == 2)[0]
	#llIdx = np.where(markerTargets == 4)[0]
	#tIdx = np.where(markerTargets == 5)[0]
	#rlIdx = np.where(markerTargets == 6)[0]
	
	
	
	
	
	#tCount = tIdx.shape
	#print("Thumb Marker Count: {}\tSize of First: ({},{})".format(tCount, dataSplit[tIdx[0]].shape[0],dataSplit[tIdx[0]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Second: ({},{})".format(tCount, dataSplit[tIdx[1]].shape[0],dataSplit[tIdx[1]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Third: ({},{})".format(tCount, dataSplit[tIdx[2]].shape[0],dataSplit[tIdx[2]].shape[1]))


	lhIdxMin = GetMinSteps(lhIdx, dataSplit)
	rhIdxMin = GetMinSteps(rhIdx, dataSplit)
	#llIdxMin = GetMinSteps(llIdx, dataSplit)
	#tIdxMin = GetMinSteps(tIdx, dataSplit)
	#rlIdxMin = GetMinSteps(rlIdx, dataSplit)
	#minValues = [lhIdxMin, rhIdxMin, llIdxMin, tIdxMin, rlIdxMin]
	minValues = [lhIdxMin, rhIdxMin]
	#minValues

	#Truncate the data to the min size
	minValue = np.min(minValues)
	print(minValue)

	lhData = GetData(lhIdx, dataSplit, minValue)
	rhData = GetData(rhIdx, dataSplit, minValue)
	#llData = GetData(llIdx, dataSplit, minValue)
	#tData = GetData(tIdx, dataSplit, minValue)
	#rlData = GetData(rlIdx, dataSplit, minValue)
	#print("Length of tData: {}".format(len(tData)))
	#print("Length of iData: {}".format(len(iData)))
	#print("Length of mData: {}".format(len(mData)))
	#print("Length of rData: {}".format(len(rData)))
	#print("Length of pData: {}".format(len(pData)))

	#minLen = np.min([len(lhData), len(rhData), len(llData), len(tData), len(rlData)])
	minLen = np.min([len(lhData), len(rhData)])
	##Want to make sure they are balanced, so we keep the minLen values
	lhData = lhData[0:minLen]
	rhData = rhData[0:minLen]
	#llData = llData[0:minLen]
	#tData = tData[0:minLen]
	#rlData = rlData[0:minLen]


	# In[68]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1]),(minLen,1))
	#llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	#tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	#rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))
    
	#Print the sizes of each data
	#print("lhData shape: {}".format(lhData.shape))

	#markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	markerTargets = np.vstack((lhTargets, rhTargets))
	#Data = np.vstack((lhData, rhData, llData, tData, rlData))
	Data = np.vstack((lhData, rhData))
    
	#Sanity Check
	#print("Data Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=Data.shape, arg2=markerTargets.shape))

	## Shuffle the data
	Data, markerTargets = shuffle(Data, markerTargets, random_state=0)

	## Split into train and test sets
	DataTrain, DataTest, markerTargetsTrain, markerTargetsTest = train_test_split(Data, markerTargets, test_size=0.3, random_state=1)
	markerTargetsTrain.shape

	## Reshape the data for time-series processing
	## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
	DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
	DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

	## Construct the model
	LSTM_EEG = Sequential()
	LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
	LSTM_EEG.add(LSTM((50), return_sequences=False))
	LSTM_EEG.add(Dense((2),activation='sigmoid'))

	LSTM_EEG.summary()
	sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
	LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


	# In[79]:


	history = LSTM_EEG.fit(DataTrain, markerTargetsTrain, epochs=30,verbose=2, batch_size=16)


	# In[80]:


	predictionsTest = LSTM_EEG.predict(DataTest)


	# In[81]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[82]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[83]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[85]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain.shape[0] - correctCountTrain


	# In[87]:
	trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
	testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

	#Save these values into the array
	trainAccuracyNoCSP[0,dataset] = trainAcc_noCSP;
	testAccuracyNoCSP[0, dataset] = testAcc_noCSP;

	print("#################################")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
	print("#################################")
	print("#################################")


	# In[88]:


	## Applying CSP to 5F data
	#filters = CSP(lhData, rhData, llData, tData, rlData)
	filters = CSP(lhData, rhData)

	# In[90]:


	filtersArray = np.asarray(filters)


	# In[91]:


	filtersArray.shape


	# In[92]:


	lhData_CSP = np.matmul(np.transpose(filtersArray[0]), lhData)
	rhData_CSP = np.matmul(np.transpose(filtersArray[1]), rhData)
	#llData_CSP = np.matmul(np.transpose(filtersArray[2]), llData)
	#tData_CSP = np.matmul(np.transpose(filtersArray[3]), tData)
	#rlData_CSP = np.matmul(np.transpose(filtersArray[4]), rlData)


	# In[93]:


	#Data_CSP = np.vstack((lhData_CSP, rhData_CSP, llData_CSP, tData_CSP, rlData_CSP))
	Data_CSP = np.vstack((lhData_CSP, rhData_CSP))


	# In[94]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1]),(minLen,1))
	#llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	#tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	#rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))

	#markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	markerTargets = np.vstack((lhTargets, rhTargets))

	# In[95]:


	## Shuffle the data
	Data_CSP, markerTargets_CSP = shuffle(Data_CSP, markerTargets, random_state=0)
	## Split into train and test sets
	DataTrain_CSP, DataTest_CSP, markerTargetsTrain_CSP, markerTargetsTest_CSP = train_test_split(Data_CSP, markerTargets_CSP, test_size=0.3, random_state=1)

	##Use the same model to train and test
	history_CSP = LSTM_EEG.fit(DataTrain_CSP, markerTargetsTrain_CSP, epochs=30,verbose=2, batch_size=16)


	# In[96]:


	predictionsTest = LSTM_EEG.predict(DataTest_CSP)


	# In[97]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[98]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[99]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[100]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest_CSP.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain_CSP)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain_CSP.shape[0] - correctCountTrain

	#Computing the accuracy
	trainAcc_wCSP = (correctCountTrain*100/DataTrain_CSP.shape[0]);
	testAcc_wCSP = (correctCountTest*100/DataTest_CSP.shape[0]);

	#Save these values into the array
	trainAccuracyWithCSP[0,dataset] = trainAcc_wCSP;
	testAccuracyWithCSP[0, dataset] = testAcc_wCSP;

	print("#################################")
	print("### RESULTS AFTER APPLYING CSP ##")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_wCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_wCSP))
	print("#################################")
	print("#################################")




[[ -0.    -0.    -0.   ...  23.8   10.74   0.76]
 [ -0.    -0.    -0.   ... -28.4  -37.39 -47.95]
 [ -0.    -0.    -0.   ...   4.31   5.51   3.66]
 ...
 [ -0.    -0.    -0.   ...   6.99   5.57   3.68]
 [ -0.    -0.    -0.   ...  -8.31  -9.34  -7.32]
 [ -0.    -0.    -0.   ...  -6.    -5.99  -4.9 ]]
202
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 21, 100)           121200    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102       
Total params: 151,502
Trainable params: 151,502
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
27/27 - 0s - loss: 0.6882 - accuracy: 0.5498
Epoch 2/30
27/27 -

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


27/27 - 0s - loss: 0.7253 - accuracy: 0.6919
Epoch 2/30
27/27 - 0s - loss: 0.4007 - accuracy: 0.8365
Epoch 3/30
27/27 - 0s - loss: 0.2362 - accuracy: 0.9123
Epoch 4/30
27/27 - 0s - loss: 0.1559 - accuracy: 0.9455
Epoch 5/30
27/27 - 0s - loss: 0.1096 - accuracy: 0.9550
Epoch 6/30
27/27 - 0s - loss: 0.0935 - accuracy: 0.9645
Epoch 7/30
27/27 - 0s - loss: 0.0997 - accuracy: 0.9597
Epoch 8/30
27/27 - 0s - loss: 0.1065 - accuracy: 0.9597
Epoch 9/30
27/27 - 0s - loss: 0.1067 - accuracy: 0.9597
Epoch 10/30
27/27 - 0s - loss: 0.0833 - accuracy: 0.9716
Epoch 11/30
27/27 - 0s - loss: 0.0567 - accuracy: 0.9858
Epoch 12/30
27/27 - 0s - loss: 0.0422 - accuracy: 0.9858
Epoch 13/30
27/27 - 0s - loss: 0.0318 - accuracy: 0.9905
Epoch 14/30
27/27 - 0s - loss: 0.0298 - accuracy: 0.9905
Epoch 15/30
27/27 - 0s - loss: 0.0287 - accuracy: 0.9905
Epoch 16/30
27/27 - 0s - loss: 0.0351 - accuracy: 0.9929
Epoch 17/30
27/27 - 0s - loss: 0.0092 - accuracy: 1.0000
Epoch 18/30
27/27 - 0s - loss: 0.0054 - accuracy: 0

28/28 - 0s - loss: 0.6997 - accuracy: 0.5421
Epoch 2/30
28/28 - 0s - loss: 0.6086 - accuracy: 0.7267
Epoch 3/30
28/28 - 0s - loss: 0.5261 - accuracy: 0.7449
Epoch 4/30
28/28 - 0s - loss: 0.4604 - accuracy: 0.7927
Epoch 5/30
28/28 - 0s - loss: 0.3843 - accuracy: 0.8519
Epoch 6/30
28/28 - 0s - loss: 0.3422 - accuracy: 0.8428
Epoch 7/30
28/28 - 0s - loss: 0.2556 - accuracy: 0.8975
Epoch 8/30
28/28 - 0s - loss: 0.2666 - accuracy: 0.8884
Epoch 9/30
28/28 - 0s - loss: 0.2393 - accuracy: 0.9021
Epoch 10/30
28/28 - 0s - loss: 0.2228 - accuracy: 0.9226
Epoch 11/30
28/28 - 0s - loss: 0.1611 - accuracy: 0.9339
Epoch 12/30
28/28 - 0s - loss: 0.1496 - accuracy: 0.9339
Epoch 13/30
28/28 - 0s - loss: 0.1167 - accuracy: 0.9658
Epoch 14/30
28/28 - 0s - loss: 0.0890 - accuracy: 0.9658
Epoch 15/30
28/28 - 0s - loss: 0.1219 - accuracy: 0.9544
Epoch 16/30
28/28 - 0s - loss: 0.1473 - accuracy: 0.9362
Epoch 17/30
28/28 - 0s - loss: 0.2179 - accuracy: 0.9089
Epoch 18/30
28/28 - 0s - loss: 0.1476 - accuracy: 0

28/28 - 0s - loss: 0.0409 - accuracy: 0.9886
Epoch 16/30
28/28 - 0s - loss: 0.0532 - accuracy: 0.9863
Epoch 17/30
28/28 - 0s - loss: 0.0563 - accuracy: 0.9818
Epoch 18/30
28/28 - 0s - loss: 0.0512 - accuracy: 0.9841
Epoch 19/30
28/28 - 0s - loss: 0.0426 - accuracy: 0.9863
Epoch 20/30
28/28 - 0s - loss: 0.0480 - accuracy: 0.9841
Epoch 21/30
28/28 - 0s - loss: 0.1099 - accuracy: 0.9567
Epoch 22/30
28/28 - 0s - loss: 0.1395 - accuracy: 0.9453
Epoch 23/30
28/28 - 0s - loss: 0.1532 - accuracy: 0.9385
Epoch 24/30
28/28 - 0s - loss: 0.0966 - accuracy: 0.9704
Epoch 25/30
28/28 - 0s - loss: 0.0560 - accuracy: 0.9795
Epoch 26/30
28/28 - 0s - loss: 0.0436 - accuracy: 0.9863
Epoch 27/30
28/28 - 0s - loss: 0.0277 - accuracy: 0.9954
Epoch 28/30
28/28 - 0s - loss: 0.0133 - accuracy: 0.9954
Epoch 29/30
28/28 - 0s - loss: 0.0047 - accuracy: 1.0000
Epoch 30/30
28/28 - 0s - loss: 0.0022 - accuracy: 1.0000
#################################
### RESULTS AFTER APPLYING CSP ##
################################

...

28/28 - 0s - loss: 0.0141 - accuracy: 0.9977
Epoch 20/30
28/28 - 0s - loss: 0.0097 - accuracy: 0.9977
Epoch 21/30
28/28 - 0s - loss: 0.0058 - accuracy: 1.0000
Epoch 22/30
28/28 - 0s - loss: 0.0030 - accuracy: 1.0000
Epoch 23/30
28/28 - 0s - loss: 0.0020 - accuracy: 1.0000
Epoch 24/30
28/28 - 0s - loss: 0.0016 - accuracy: 1.0000
Epoch 25/30
28/28 - 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 26/30
28/28 - 1s - loss: 0.0011 - accuracy: 1.0000
Epoch 27/30
28/28 - 0s - loss: 9.7031e-04 - accuracy: 1.0000
Epoch 28/30
28/28 - 0s - loss: 8.6725e-04 - accuracy: 1.0000
Epoch 29/30
28/28 - 0s - loss: 7.8957e-04 - accuracy: 1.0000
Epoch 30/30
28/28 - 0s - loss: 7.3122e-04 - accuracy: 1.0000
#################################
### RESULTS AFTER APPLYING CSP ##
#################################
Training Performance:
Correct MI Prediction: 439
Incorrect MI Prediction: 0
Percent Accuracy: 100.000%
#################################
#################################
Testing Performance:
Correct MI Predict

28/28 - 0s - loss: 0.1181 - accuracy: 0.9544
Epoch 14/30
28/28 - 0s - loss: 0.0852 - accuracy: 0.9613
Epoch 15/30
28/28 - 0s - loss: 0.0466 - accuracy: 0.9841
Epoch 16/30
28/28 - 0s - loss: 0.0510 - accuracy: 0.9818
Epoch 17/30
28/28 - 0s - loss: 0.0328 - accuracy: 0.9863
Epoch 18/30
28/28 - 0s - loss: 0.1219 - accuracy: 0.9567
Epoch 19/30
28/28 - 0s - loss: 0.1351 - accuracy: 0.9522
Epoch 20/30
28/28 - 0s - loss: 0.0885 - accuracy: 0.9704
Epoch 21/30
28/28 - 0s - loss: 0.0600 - accuracy: 0.9749
Epoch 22/30
28/28 - 0s - loss: 0.0259 - accuracy: 0.9909
Epoch 23/30
28/28 - 0s - loss: 0.0187 - accuracy: 0.9932
Epoch 24/30
28/28 - 0s - loss: 0.0419 - accuracy: 0.9841
Epoch 25/30
28/28 - 0s - loss: 0.0611 - accuracy: 0.9795
Epoch 26/30
28/28 - 0s - loss: 0.0217 - accuracy: 0.9932
Epoch 27/30
28/28 - 0s - loss: 0.0498 - accuracy: 0.9841
Epoch 28/30
28/28 - 0s - loss: 0.0300 - accuracy: 0.9886
Epoch 29/30
28/28 - 0s - loss: 0.0650 - accuracy: 0.9727
Epoch 30/30
28/28 - 0s - loss: 0.0680 - acc

Epoch 26/30
28/28 - 0s - loss: 5.9338e-04 - accuracy: 1.0000
Epoch 27/30
28/28 - 0s - loss: 5.5780e-04 - accuracy: 1.0000
Epoch 28/30
28/28 - 0s - loss: 5.2622e-04 - accuracy: 1.0000
Epoch 29/30
28/28 - 0s - loss: 4.9831e-04 - accuracy: 1.0000
Epoch 30/30
28/28 - 0s - loss: 4.7413e-04 - accuracy: 1.0000
#################################
### RESULTS AFTER APPLYING CSP ##
#################################
Training Performance:
Correct MI Prediction: 439
Incorrect MI Prediction: 0
Percent Accuracy: 100.000%
#################################
#################################
Testing Performance:
Correct MI Prediction: 161
Incorrect MI Prediction: 28
Percent Accuracy: 85.185%
#################################
#################################
[[-0. -0. -0. ...  0.  0. -0.]
 [-0. -0. -0. ...  0.  0. -0.]
 [-0. -0. -0. ... -0.  0.  0.]
 ...
 [-0. -0. -0. ...  0.  0. -0.]
 [-0. -0. -0. ...  0. -0. -0.]
 [-0. -0. -0. ...  0.  0. -0.]]
202
Model: "sequential_11"
_________________________________

42/42 - 1s - loss: 0.1667 - accuracy: 0.9280
Epoch 23/30
42/42 - 1s - loss: 0.1846 - accuracy: 0.9220
Epoch 24/30
42/42 - 1s - loss: 0.1868 - accuracy: 0.9205
Epoch 25/30
42/42 - 1s - loss: 0.1667 - accuracy: 0.9355
Epoch 26/30
42/42 - 1s - loss: 0.1396 - accuracy: 0.9490
Epoch 27/30
42/42 - 1s - loss: 0.0908 - accuracy: 0.9655
Epoch 28/30
42/42 - 1s - loss: 0.0681 - accuracy: 0.9790
Epoch 29/30
42/42 - 1s - loss: 0.0645 - accuracy: 0.9790
Epoch 30/30
42/42 - 1s - loss: 0.0702 - accuracy: 0.9715
#################################
#################################
Training Performance:
Correct MI Prediction: 647
Incorrect MI Prediction: 20
Percent Accuracy: 97.001%
#################################
#################################
Testing Performance:
Correct MI Prediction: 158
Incorrect MI Prediction: 129
Percent Accuracy: 55.052%
#################################
#################################
Epoch 1/30
42/42 - 1s - loss: 0.6860 - accuracy: 0.7136
Epoch 2/30
42/42 - 1s - loss: 0.3

[[-0.   -0.   -0.   ... 10.56  1.81 -0.91]
 [-0.   -0.   -0.   ... 36.47 33.84 27.07]
 [-0.   -0.   -0.   ... -7.12 -3.06 -5.77]
 ...
 [-0.   -0.   -0.   ...  6.48  6.73  4.6 ]
 [-0.   -0.   -0.   ...  4.61  5.54  5.48]
 [-0.   -0.   -0.   ... -5.87 -3.91 -4.09]]
202
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 21, 100)           121200    
_________________________________________________________________
lstm_29 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 102       
Total params: 151,502
Trainable params: 151,502
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
28/28 - 0s - loss: 0.7012 - accuracy: 0.4853
Epoch 2/30
28/28 - 0s - loss: 0.6492 - accuracy: 0.65

In [17]:
fileNames

['CLASubjectA1601083StLRHand.mat',
 'CLASubjectB1510193StLRHand.mat',
 'CLASubjectB1510203StLRHand.mat',
 'CLASubjectB1512153StLRHand.mat',
 'CLASubjectC1511263StLRHand.mat',
 'CLASubjectC1512163StLRHand.mat',
 'CLASubjectC1512233StLRHand.mat',
 'CLASubjectD1511253StLRHand.mat',
 'CLASubjectE1512253StLRHand.mat',
 'CLASubjectE1601193StLRHand.mat',
 'CLASubjectE1601223StLRHand.mat',
 'CLASubjectF1509163StLRHand.mat',
 'CLASubjectF1509173StLRHand.mat',
 'CLASubjectF1509283StLRHand.mat']

In [18]:
testAccuracyNoCSP

array([[54.94505495, 52.38095238, 51.85185185, 59.25925926, 67.72486772,
        58.73015873, 61.37566138, 51.06382979, 64.02116402, 56.61375661,
        65.95744681, 55.05226481, 50.54347826, 52.35602094]])

In [19]:
testAccuracyWithCSP

array([[80.21978022, 87.3015873 , 86.24338624, 71.42857143, 81.48148148,
        84.65608466, 83.5978836 , 78.72340426, 84.12698413, 85.18518519,
        90.42553191, 91.63763066, 91.30434783, 87.43455497]])

In [20]:
trainAccuracyNoCSP

array([[100.        ,  74.03189066,  99.08883827, 100.        ,
        100.        , 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        ,  97.00149925,
         96.72897196,  97.74266366]])

In [21]:
trainAccuracyWithCSP

array([[100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
        100., 100., 100.]])

In [22]:
combinedArray = np.transpose(np.asarray(np.vstack((trainAccuracyNoCSP,testAccuracyNoCSP,trainAccuracyWithCSP, testAccuracyWithCSP))))

In [23]:
combinedArray.shape

(14, 4)

In [24]:
np.savetxt('CLAperSubject.csv', combinedArray, delimiter=',')

TypeError: Mismatch between array dtype ('<U40') and format specifier ('%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e')